This notebook runs the list-learning behavioral analyses

## Import libraries

In [ ]:
import quail
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib
from scipy.spatial.distance import cdist
%matplotlib inline

## Settings and parameters

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
datadir = '../../../data/processed/'
palette = 'bone'

## Load data

In [ ]:
video_events = np.load(datadir+'video_event_model.npy')
recall_events = np.load(datadir+'recall_event_models.npy')

## Convert to egg format

In [ ]:
pres = [[[{'item' : i, 'topics' : m} for i, m in enumerate(video_events)]] for sub in range(17)]

rec = []
for r in recall_events:
    items = list(np.argmax(1 - cdist(video_events, r), 0).astype(int))
    rec.append([[{'item' : item, 'topics' : t} for item, t in zip(items, r)]])

## Make the egg

In [ ]:
egg = quail.Egg(pres=pres, rec=rec)
egg.info()

## Run probability of first recall, lagcrp and serial position curve analyses, and create a figure

In [ ]:
fig, axarr = plt.subplots(1, 3)
fig.set_size_inches(10, 3)

analysis = ['pfr', 'lagcrp', 'spc']
res = {}
for ax, a in zip(axarr, analysis):
    res[a] = egg.analyze(a, match='best', features=['topics'], 
                distance='correlation')
    res[a].plot(show=False, ax=ax, legend=False, palette=palette)
        
# make some adjustments
axarr[1].set_xlim(-10, 10)
axarr[0].set_ylabel('Probability of first recall')
axarr[1].set_ylabel('Conditional response probability')
axarr[2].set_ylabel('Proportion recalled')
axarr[0].set_xlabel('Event number')
axarr[1].set_xlabel('Relative event number')
axarr[2].set_xlabel('Event number')
plt.subplots_adjust(wspace=.45, hspace=.75)
plt.tight_layout()

# add labels
label=['A', 'B', 'C']
for p, l in zip(axarr, label):
    p.text(-.1,1.025,l,
        horizontalalignment='center',
        transform=p.transAxes,
          fontsize=12)
# plt.savefig('../figures/list_learning.pdf')
plt.show()

## Cluster analyses

In [ ]:
clustering = egg.analyze('fingerprint')
clustering.plot(ylim=[0, 1.2], palette=palette)

## SEM of clustering scores

In [ ]:
sem = clustering.get_data().std() / np.sqrt(17)
print(clustering.get_data().mean())
print(sem)
print(clustering.get_data().mean() + sem)
clustering.get_data().mean() - sem

## Create a dataframe containing the analysis results

In [ ]:
ks = np.array([8, 14, 11, 9, 14, 10, 17, 17, 10, 20, 19, 25, 27, 14, 10, 14, 23])
hand_rec = np.array([27, 24, 32, 33, 32, 39, 30, 39, 28, 40, 34, 38, 47, 38, 27, 37, 39])
auto_rec = res['spc'].data.mean(1).values
df = pd.DataFrame(np.array([clustering.data.temporal.values, clustering.data.topics.values, ks, hand_rec, auto_rec]).T, 
                  columns=['Temporal clustering', 'Semantic clustering', 'Number of events (k)', 'Memory performance - hand', 
                           'Memory performance - auto'])

## Correlation between hand annotated memory and memory estimated from the model across subjects

In [ ]:
sns.jointplot(x=df['Memory performance - hand'], y=df['Memory performance - auto'], kind='reg', 
              space=False, color='k')

## Correlation between hand annotated memory and k (number of events) across subjects

In [ ]:
sns.jointplot(y=df['Memory performance - hand'], x=df['Number of events (k)'], kind='reg', space=False, color='k')

## Correlation between hand annotated memory and temporal clustering

In [ ]:
sns.jointplot(y=df['Memory performance - hand'], x=df['Temporal clustering'], kind='reg', space=False, color='k')

## Correlation between hand annotated memory and semantic clustering

In [ ]:
sns.jointplot(y=df['Memory performance - hand'], x=df['Semantic clustering'], kind='reg', space=False, color='k')